# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem. The algorithm will classify students into 2 groups, the one that will graduate and the ones that will not. A student that will not graduate will require intervention.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = len(student_data['passed'][lambda x: x=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data['passed'][lambda x: x=='no'])

# TODO: Calculate graduation rate
grad_rate = (n_passed * 100.0) / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

#Data Analysis


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None
# Added the stratify parameter as per reviewer one to match the ratio between training and test.
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, stratify=y_all,train_size=num_train, test_size=num_test, random_state=5)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

# Imbalance Analysis (Tips from the first review)
print 
print "Trainig yes to no balance:"
print y_train.value_counts()
print
print "Testing yes to no balance:"
print y_test.value_counts()
print 
print "Both the training targets and the test tragets are imbalance. There is ~ 3:1 ratio of yes and no"

from sklearn.metrics import f1_score
print "\nF1 score for predicting all 'yes': {:.4f}".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, pos_label='yes', average='binary'))

print "We should get a different F1 score for our models, otherwise it could mean we are just predicting yes everything.. "

Training set has 300 samples.
Testing set has 95 samples.

Trainig yes to no balance:
yes    201
no      99
Name: passed, dtype: int64

Testing yes to no balance:
yes    64
no     31
Name: passed, dtype: int64

Both the training targets and the test tragets are imbalance. There is ~ 3:1 ratio of yes and no

F1 score for predicting all 'yes': 0.8030
We should get a different F1 score for our models, otherwise it could mean we are just predicting yes everything.. 


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
### Gaussian Naive Bayes
**Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**
Document classification is an example of the used of the ML Naive Bayes classification model. As the WikiPidia site "https://en.wikipedia.org/wiki/Document_classification" stays, one of the common methods to automatically classify library documents is using supervise learning and one of the technic is Naive Bayes.
**What are the strengths of the model; when does it perform well?**
Performs well with high dimesional data sets.
Performs well with small training data sets.
Performs well because of its independence features property, all features are tretaed the same (weightless).
**What are the weaknesses of the model; when does it perform poorly?**
It can't learn dependencies of features. Example occurence of "A" affects positively an outcome and the occurrence "B" also does it, but ocurrence of "A" and "B" at the same time affects it negatively.
Because it doesn't related features is not good for continues problems.
**What makes this model a good candidate for the problem, given what you know about the data?**
Few training points 300 points
Clasification problem (discrete output not continues)

### Decision Trees
**Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**
Decision trees have helped in star-galaxy classification "http://adsabs.harvard.edu/abs/1995AJ....109.2401W"
**What are the strengths of the model; when does it perform well?**
Decision trees models are human readable so it can be interpreted. One can see how the classifier gets to the prediction by looking at the tree.
Contrary to Naive Bayes, decision trees models the relation between the features.
**What are the weaknesses of the model; when does it perform poorly?**
Performs poorly when the problem has a large set of features and training data is not enought. This causes the classifier to overfit and not generalize well, if it is presented with new data.
Once the model has learned, it can't be improved by adding new evidence, it has to be re-trained. This is caused because it is an eager leaner, once it is trained it creates a general function to predic future data. The function depends on what the model learned from the training data. New evidence that wasn't on the training data can negatively impact future prediction unless the classifier be trained again including the new data.
**What makes this model a good candidate for the problem, given what you know about the data?**
Decision trees models perform really well in classification problems.
If the data is not lianry separatable still performs well.
It takes in cosideration the dependent relation between features.


### Support Vector Machines (SVM) ###
**Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**
SVM is used in the classification of ECG signals (http://www.iaeng.org/IJAM/issues_v36/issue_2/IJAM_36_2_4.pdf). Other ares in bioinformatic that also benefits from SVM is the gene classification, molecule classification, etc.
**What are the strengths of the model; when does it perform well?**
Its performs well in non lineary separable classes.
Works best in binary classifications. Using kernels can classify objects by separated them on a higher dimesion.
**What are the weaknesses of the model; when does it perform poorly?**
No good with noise class overlapping.
Don't work well when there a lot data points.
Higher resources cost and slower prediction than other classifiers like Decision Trees.
Ensemble models are very complex and can have high computational cost.
**What makes this model a good candidate for the problem, given what you know about the data?**
Few data points, Binary classification problem.




### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
# Even if I explored RF I am not going to consider it. Second reviewer recommended adding this parameter random state for
# all the classifier that support it
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier(random_state = 42)
clf_C = RandomForestClassifier(random_state = 42)  
clf_D = SVC(random_state = 42)

# TODO: Set up the training set sizes

X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
# As per review recommendation I simplified  the code using loops.
# TODO: loop thru models
for clf in [clf_A,clf_B,clf_C,clf_D]:

    # show the estimator type
    print "\n{}: \n".format(clf.__class__.__name__)

    # TODO: loop thru training sizes
    for n in [100,200,300]:

        # fit model using "n" training data points (i.e., 100, 200, or 300)
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)

# Recommended Benchmark First Review
from sklearn.metrics import f1_score
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
    f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))



GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8811.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7846.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0011 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.7985.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.8000.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0012 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.7674.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7969.

DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7481.
Training a DecisionTreeCla

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0014               |    0.0004              |   0.8811         |    0.7846       |
| 200               |    0.0011               |    0.0005              |   0.7985         |    0.8000       |
| 300               |    0.0012               |    0.0004              |   0.7674         |    0.7969       |

** Classifer 2 - Decision Trees**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0008              |        0.0002          |     1.0000       |     0.7481      |
| 200               |     0.0012              |        0.0002          |     1.0000       |     0.7692      |
| 300               |     0.0019              |        0.0002          |     1.0000       |     0.6780      |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0012            |        0.0008          |      0.8589      |     0.8077      |
| 200               |       0.0034            |        0.0025          |      0.8571      |     0.8052      |
| 300               |       0.0069            |        0.0038          |      0.8590      |     0.8163      |



## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
what single model you chose as the best model?
The first time I submitted the project the Decision Tree gave me the best scores. After understand the imbalance concept and applying the stratify paramter in the splitting of the data, the Support Vector Machine improved a lot, so I decided to change my choice to SVC. It performed way better than DT before been optimized and after. 

After comparing the f1 score of predicting all the values as "yes" in the test sets with the SVM score, the SVM score is higher which means that is predicting the "no" classess correctly.


Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
\>>>Available Data Properties:
Giving the small sample of data, high dimensionality (30 features) and  potential features depndency; the appropiate model is SVM.  DT came close in second, but even if the computational cost of training and the prediction time of SVM if higher than DT, once the SVM is optimized using GridSearch it produces better scores than the optimized version of DT. 

\>>>limited resources:
We only have 395 data points in total and because of cross validation what is left for training is a very small sample set. Based on the scores it looks like the features have some small degree of dependency. 

\>>>cost:
DT seems to be the one that has less computational cost and better response time. But SVM produces better results. 

\>>>performance:
Of all three models SVM produced the best test scores. DT came close in second.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
Support Vector Machine is a classification computational algorithm. By classification we means the action of grouping objects according to shared qualities or characteristics. The human brain perform such task by separating objects with similar characteristics. Humans usually separate the objects apart by creating enough physical space between them so a second observer can easily perceive them as 2 separate classes of the same object. Example of this is a table with ping pong balls of 2 different colors (blue and purple) if we separate them but the space we leave between the 2 groups is not larger enough an observer requires to be close to the table to differentiate them, so the more space we put between the 2 type of balls the better the observer will be able to distinguish the 2 classe. It is important to notice that the space between ball of different colors will be the distance between the closest blue and purple color balls.

A support Vector Machine is a programatically way to instruct a computer to do exactly what a human does but instead of doing it in a physical world it is done in a digital one. In the digital world and using the same example of the previous paragraph, the color balls become just color dots in a  screen, SVM using a coordinate system map all the dots location on the screen and them draws lines over the closest 2 dots of different colors. The distance between those 2 lines is equivalent to the space humans produce by separting the balls on the table. That space is called the "marging". Same as in the physical world the larger the marging the better an observer can separate the type of objects. 

Going back the the example of the balls in a physical world, let us say that the blue balls are encircling the purple ones. A person has to be more resourceful and separate them without re-arrange them. It does it by creating a surronding space between the 2 colors. Computers can't do the same using the line trick with learned before so what a computer does is add more space dimensions to do the trick. If you have been in a movie theather recently you probably has watch a 3D movie. In a 3D movie you see all kind of objects getting out of screen while others stays behind. You know that the movie screen is flat so there is trick done by the camera that produces that effect. That is exacly what the SVM program does it raises the purple dots leaving behind the blues and now the 2 dots types are separated in a 3D space. After the separation is done SVM slices both groups with what will look like in the real world as a sheet of paper, that sheet of paper is called a "hyperplane". If all the sudden the 3D dissapears this hyperplane will left behind a circular line that separate the colors dots. This trick that projects the dots in a 3D dimesion and creates the circular separation is achived by adding to KVM program a mathematical helper function called the kernel.  

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [25]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

# TODO: Create the parameters list you wish to tune
#parameters = {'max_depth':range(1,20),'min_samples_leaf':range(5,10)}
parameters = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}

# TODO: Initialize the classifier
#clf = DecisionTreeClassifier()
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer'
def performance_metric(y_true, y_predict):
    from sklearn.metrics import f1_score
    score = f1_score(y_true,y_predict,pos_label="yes")
    return score

f1_scorer = make_scorer(performance_metric)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf,parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

print

from IPython.display import display
display(pd.DataFrame(grid_obj.grid_scores_))



Made predictions in 0.0046 seconds.
Tuned model has a training F1 score of 0.8206.
Made predictions in 0.0015 seconds.
Tuned model has a testing F1 score of 0.8299.



,parameters,mean_validation_score,cv_validation_scores
0,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0.001}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
1,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0.0001}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
2,"{u'kernel': u'rbf', u'C': 10, u'gamma': 0.001}",0.807438,"[0.805031446541, 0.802469135802, 0.814814814815]"
3,"{u'kernel': u'rbf', u'C': 10, u'gamma': 0.0001}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
4,"{u'kernel': u'rbf', u'C': 100, u'gamma': 0.001}",0.759138,"[0.773333333333, 0.775510204082, 0.728571428571]"
5,"{u'kernel': u'rbf', u'C': 100, u'gamma': 0.0001}",0.811449,"[0.805031446541, 0.807228915663, 0.822085889571]"
6,"{u'kernel': u'rbf', u'C': 1000, u'gamma': 0.001}",0.747045,"[0.709219858156, 0.780141843972, 0.751773049645]"
7,"{u'kernel': u'rbf', u'C': 1000, u'gamma': 0.0001}",0.787106,"[0.797385620915, 0.794701986755, 0.769230769231]"


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
What is the final model's F1 score for training and testing?
Training: 0.8206
Test: 0.8299

How does that score compare to the untuned model?
After tuned the model we can see a significant improvement on the test score.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.